In [1]:
from bs4 import BeautifulSoup
import requests
#!pip install pymongo
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/melissafraas/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'


browser.visit(url)
browser.is_element_present_by_css('div.list_text', wait_time=3)

response = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response, 'html.parser')
soup

<html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
<title>News - Mars Exploration Program</title>
</head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</spa

In [5]:
soup2=soup.select_one('div.list_text')

In [6]:
news_title = soup2.find('div', class_="content_title").get_text()
news_title

"3 Things We've Learned From NASA's Mars InSight "

In [7]:
news_p=soup2.find('div', class_="article_teaser_body").get_text()
news_p

'Scientists are finding new mysteries since the geophysics mission landed two years ago.'

# Featured Image Scraping

In [8]:
# Featured space image 
#https://spaceimages-mars.com

   
    
featured_image_url= "https://spaceimages-mars.com/"
browser.visit(featured_image_url)
response = browser.html

soup = BeautifulSoup(response, 'html.parser') 
button = soup.find('button', class_='btn btn-outline-light')
images = soup.find_all('img')
featured_image=soup.find('img', class_='headerimage fade-in')['src']
featured_image

#put together final url for featured image only
photo_url= featured_image_url + featured_image
photo_url


'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Mars Facts Scraping

In [9]:
#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

mars_facts_url = "https://galaxyfacts-mars.com"
tables = pd.read_html(mars_facts_url)
tables
df = tables [0]
df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n      <th>2</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Mars - Earth Comparison</td>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</

# Mars Hemispheres

In [11]:
hemispheres_url = "https://marshemispheres.com/"
browser.visit(hemispheres_url)
#the webpage contains a list of links that need to be clicked in order to view full image.

# click_links=soup.find('a')['href']
# click_links
hemisphere_links = {}
hemisphere_list = []
click_links = browser.find_by_css('a.product-item img')
#[print(x) for x in click_links]
for i in range(len(click_links)):
    browser.find_by_css('a.product-item img')[i].click()
    sample = browser.links.find_by_text("Sample").first
    hemisphere_links['img_url'] = sample['href']
    hemisphere_links['img_title'] = browser.find_by_css('h2.title').text
    hemisphere_list.append(hemisphere_links)
    browser.back()
print(hemisphere_list)


[{'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'img_title': 'Valles Marineris Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'img_title': 'Valles Marineris Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'img_title': 'Valles Marineris Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'img_title': 'Valles Marineris Hemisphere Enhanced'}]


In [12]:
hemisphere_list[2]

{'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
 'img_title': 'Valles Marineris Hemisphere Enhanced'}

In [14]:
browser.quit()